In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

In [0]:
entities = spark.sql(f"SELECT WatermarkColumn, TargetTable FROM {catalog}.{schema}.file_master_config WHERE EnableFlag=1").collect()

for e in entities:
    cnt = spark.sql(f"SELECT COUNT(*) FROM {catalog}.{schema}.{e['TargetTable']}").first()[0]
    
    if cnt > 0:
        spark.sql(
            f"""
            UPDATE {catalog}.{schema}.file_master_config 
            SET 
                LPWatermarkValue = (SELECT MAX({e['WatermarkColumn']}) FROM {catalog}.{schema}.{e['TargetTable']})
            WHERE 
                TargetTable = '{e["TargetTable"]}'
            """
        )
